In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [9]:
def bayesian_ridge_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    n_iter=300,
    alpha_1=1e-6,
    alpha_2=1e-6,
    lambda_1=1e-6,
    lambda_2=1e-6,
    compute_scores=True,
    random_state=None
):
    """
    Bayesian Ridge Regression for time series forecasting with:
    - Probabilistic uncertainty estimates
    - Automatic complexity control
    - Lag features only
    """
    # Create directory structure
    os.makedirs('BayesianRidgeRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features
    df = pd.DataFrame({'y': series.astype(float)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(float)
    df = df.dropna()

    # 3. Train-Test Split
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model Training with Bayesian Ridge
    model = BayesianRidge(
        n_iter=n_iter,
        alpha_1=alpha_1,
        alpha_2=alpha_2,
        lambda_1=lambda_1,
        lambda_2=lambda_2,
        compute_score=compute_scores,
    )
    model.fit(X_train, y_train)

    # 5. Forecasting with Uncertainty
    pred_test, pred_test_std = model.predict(X_test, return_std=True)

    # 6. Inverse Differencing
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 7. Create forecast DataFrame with uncertainty
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast_mean': pred_test,
        'forecast_std': pred_test_std
    })

    # 8. Save forecasts
    forecast_csv_path = os.path.join('BayesianRidgeRegression', 'bayesian_forecasts.csv')
    write_header = not os.path.exists(forecast_csv_path)
    forecast_df.to_csv(forecast_csv_path, mode='a', header=write_header, index=False)

    # 9. Calculate metrics and uncertainty statistics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast_mean']))
    log_marginal_likelihood = model.scores_[-1] if compute_scores else np.nan
    
    # Calculate coefficient statistics
    coef_means = model.coef_
    coef_stds = np.sqrt(np.diag(model.sigma_))
    
    metrics_df = pd.DataFrame([{
        'district': district_name,
        'rmse': rmse,
        'log_marginal_likelihood': log_marginal_likelihood,
        'coefficient_means': coef_means.tolist(),
        'coefficient_stds': coef_stds.tolist(),
        'intercept_mean': model.intercept_,
        'intercept_std': np.nan,
        'effective_params': model.lambda_ / model.alpha_,
        'differencing': d,
        'n_iter': n_iter
    }])

    metrics_csv_path = os.path.join('BayesianRidgeRegression', 'bayesian_metrics.csv')
    write_header_metrics = not os.path.exists(metrics_csv_path)
    metrics_df.to_csv(metrics_csv_path, mode='a', header=write_header_metrics, index=False)

    # 10. Plotting with uncertainty bands
    plt.figure(figsize=(12, 6))
    plt.plot(original_series.index, original_series.values, label='Original Series')
    plt.plot(forecast_df['date'], forecast_df['forecast_mean'], label='Mean Forecast', color='darkorange')
    plt.fill_between(
        forecast_df['date'],
        forecast_df['forecast_mean'] - 1.96*forecast_df['forecast_std'],
        forecast_df['forecast_mean'] + 1.96*forecast_df['forecast_std'],
        color='orange',
        alpha=0.2,
        label='95% Confidence'
    )
    plt.title(
        f'Bayesian Ridge Forecast for {district_name}\n'
        f'RMSE: {rmse:.2f} | Log ML: {log_marginal_likelihood:.2f}'
    )
    plt.legend()
    plt.savefig(os.path.join('BayesianRidgeRegression', f'bayesian_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model': model,
        'coef_means': coef_means,
        'coef_stds': coef_stds,
        'intercept_mean': model.intercept_,
        'intercept_std': np.nan
    }

In [ ]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I1"].asfreq('MS')
        
        results = bayesian_ridge_regression_lags_only(
            ts,
            district,
            max_lags=4,
            n_iter=500,
            alpha_1=1e-5,
            lambda_1=1e-3
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
     district        rmse  log_marginal_likelihood  \
0  AHMEDNAGAR  547.094447              -269.217875   

                                   coefficient_means  \
0  [-0.00014123796678755662, -2.204828891672853e-...   

                                    coefficient_stds  intercept_mean  \
0  [0.012739564876249108, 0.012740198082995886, 0...     6907.043918   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      1.720346e+09             0     500  

=== Forecast Data ===
              district       date  actual  forecast_mean  forecast_std
date                                                                  
2020-07-01  AHMEDNAGAR 2020-07-01  6478.0    6893.374185    559.060073
2020-08-01  AHMEDNAGAR 2020-08-01  5975.0    6892.695115    560.033636
2020-09-01  AHMEDNAGAR 2020-09-01  6550.0    6893.660781    557.179375
2020-10-01  AHMEDNAGAR 2020-10-01  6471.0    6894.402562    555.110651
2020-11-01  AHMEDNAGAR 2020-11-01  6107.0    6895.

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
   district        rmse  log_marginal_likelihood  \
0  AMRAVATI  426.503318              -247.782894   

                                   coefficient_means  \
0  [0.23000123180310356, -0.11171823602840758, -0...   

                                    coefficient_stds  intercept_mean  \
0  [0.12723276144892515, 0.12913136571165096, 0.1...     4020.051024   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      1.992296e+06             0     500  

=== Forecast Data ===
            district       date  actual  forecast_mean  forecast_std
date                                                                
2020-07-01  AMRAVATI 2020-07-01  3459.0    3803.632266    958.485216
2020-08-01  AMRAVATI 2020-08-01  3049.0    3722.250701    930.657371
2020-09-01  AMRAVATI 2020-09-01  3325.0    3696.561622    879.788002
2020-10-01  AMRAVATI 2020-10-01  3191.0    3867.660075    851.144841
2020-11-01  AMRAVATI 2020-11-01  3502.0    3873.343765    805.8808

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


...

=== Metrics ===
  district        rmse  log_marginal_likelihood  \
0   WASHIM  148.618951              -234.481868   

                                   coefficient_means  \
0  [0.2547399861941074, -0.2425647374619671, 0.08...   

                                    coefficient_stds  intercept_mean  \
0  [0.1369639989819333, 0.14091457760391307, 0.13...      1769.57716   

   intercept_std  effective_params  differencing  n_iter  
0            NaN     578549.771337             0     500  

=== Forecast Data ===
           district       date  actual  forecast_mean  forecast_std
date                                                               
2020-07-01   WASHIM 2020-07-01  1629.0    1613.352895    457.250352
2020-08-01   WASHIM 2020-08-01  1502.0    1679.792431    455.400971
2020-09-01   WASHIM 2020-09-01  1599.0    1603.190002    444.327916
2020-10-01   WASHIM 2020-10-01  1501.0    1699.102234    431.429055
2020-11-01   WASHIM 2020-11-01  1453.0    1632.067308    426.992480
=== Me

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
          district        rmse  log_marginal_likelihood  \
0  MUMBAI SUBURBAN  547.126116               -45.383661   

                                   coefficient_means  \
0  [0.17370178045390328, 0.20029270664928311, 0.1...   

                                    coefficient_stds  intercept_mean  \
0  [0.17819497277613575, 0.17831596175993802, 0.1...     4419.859653   

   intercept_std  effective_params  differencing  n_iter  
0            NaN      1.431852e+06             0     500  

=== Forecast Data ===
                   district       date   actual  forecast_mean  forecast_std
date                                                                        
2021-02-01  MUMBAI SUBURBAN 2021-02-01  13444.0   13771.678830   1838.051175
2021-03-01  MUMBAI SUBURBAN 2021-03-01  13294.0   13994.942622   1834.523871


{'district': 'MUMBAI SUBURBAN',
 'forecast_df':                    district       date   actual  forecast_mean  forecast_std
 date                                                                        
 2021-02-01  MUMBAI SUBURBAN 2021-02-01  13444.0   13771.678830   1838.051175
 2021-03-01  MUMBAI SUBURBAN 2021-03-01  13294.0   13994.942622   1834.523871,
 'metrics_df':           district        rmse  log_marginal_likelihood  \
 0  MUMBAI SUBURBAN  547.126116               -45.383661   
 
                                    coefficient_means  \
 0  [0.17370178045390328, 0.20029270664928311, 0.1...   
 
                                     coefficient_stds  intercept_mean  \
 0  [0.17819497277613575, 0.17831596175993802, 0.1...     4419.859653   
 
    intercept_std  effective_params  differencing  n_iter  
 0            NaN      1.431852e+06             0     500  ,
 'model': BayesianRidge(alpha_1=1e-05, compute_score=True, lambda_1=0.001, n_iter=500),
 'coef_means': array([0.1737017

: 